In [ ]:
import numpy as np
import nltk
import re


phons = [u'AA',u'AE',u'AH',u'AO',u'AW',u'AY',u'B',u'CH',
u'D',u'DH',u'EH',u'ER',u'EY',u'F',u'G',u'HH',u'IH',u'IY',u'JH',
u'K',u'L',u'M',u'N',u'NG',u'OW',u'OY',u'P',u'R',u'S',u'SH',u'T',
u'TH',u'UH', u'UW',u'V',u'W',u'Y',u'Z',u'ZH', u'-']

phoneme_dict =   dict(zip(phons, range(40)))


#removes stresses from vowels
def strip_stresses(phoneme):
    main_phon = phoneme[:-1]
    last_letter = phoneme[-1]
    if last_letter.isnumeric():
        return main_phon
    else:
        return phoneme
def obtain_phonemes(sentence, dictionary):
    sentence_phonemes = []
    for word in sentence:
        word = word.lower()
        if word in dictionary:
            #nltk.corpus.cmudict.dict() gives a list of lists, giving both
            #the american and english pronounciations
            phonemes = dictionary[word][0]
            stripped_phonemes = map(strip_stresses, phonemes)
            sentence_phonemes.extend(stripped_phonemes)
        else:
            #print "{} is not in the dictionary, can't get phonemes!".format(word)
            return None
    return sentence_phonemes

D = nltk.corpus.cmudict.dict()
D["what"]
R = {}

# assert len(D.items()) ==len(set([" ".join([strip_stresses(phon) for phon in h]) for homophones in D.values() for h in homophones]))
for word, hs in D.items():
    for h in hs:
        stripped = [strip_stresses(phon) for phon in h]
        phon_string = " ".join(stripped)
        if R.get(phon_string):
            R[phon_string].append(word)
        else:
            R[phon_string] = [word]
for a, b in R.items():
    if a not in [" ".join([strip_stresses(p) for p in D[x][0]]) for x in ["what", "is", "the", "phone", "number"] ]:
        del R[a]
print "R has {} keys".format(len(R.keys()))


def stimes(*factors):
    if 0. in factors:
        return 0.
    else: 
        return np.exp(np.sum(np.log(factors)))
def sexp(base, exp):
    if base ==0.:
        return 0.
    else:
        return np.exp(exp * np.log(base))
def sexp_arr(base, exps):
    
    return [0. if b==0. else np.exp(np.log(b) * e) for b,e in zip(base,exps) ]
def sdiv(a,b):
    if a == 0.:
        return 0.
    return np.exp(np.log(a)-np.log(b))

def lm_raw(w):
    
    return np.exp(0.)
# Compute the transition probability


def get_pr_lang_ph(y):
    return 1.

    
def top_SPs(prs, B, k, beta):
    sentences = [y.w for y in B]
    probs = np.asarray([prs[y.sh] for y in B])
    string_lens = np.asarray([len(sentence)+1 for sentence in sentences], dtype=float)
    scores = sexp_arr(probs,1./string_lens)
    #print scores
    top_k_idxs = np.argsort(scores)[-k:] # dividing makes them greater because they're negative!
    return [B[i] for i in top_k_idxs]




In [ ]:
def beam_search_coarse(acoustic_score, language_score, alpha, beta, frame_level_probs, n_paths):
    # Initialize the candidate-score dict with the empty string
    candidates = {"":0.}
    # Iterate over input timesteps and possibly grow the tree at each
    for t in range(T):
        
        # Retrieve the best paths so far
        top_candidates = dict(sorted(candidates.items(), key= lambda x: x[1])[:n_paths])
        
        # Check each path for whether growing it will improve its score
        for s,score in top_candidates.items():
            
            # We don't want to iterate over every word because there are a ton of them. 
            # It's probably better to let the phones dictate which ones we search
            # So we iterate through phoneme extensions and score them 
            # Check every extension allowed by the lm
            for word in language:
                s_new = s+" "+word
                if s_new in candidates.keys() break
                    
                new_phonemes = code_phonemes(s_new)
                new_score = acoustic_score(frame_level_probs, new_phonemes) \
                            + alpha*language_score(s_new) \
                            + beta*len(s_new.split(" "))
                if new_score >= score:
                    candidates[s_new] = new_score